# Application Example 2 

Suppose we have a `Polygon` class that has a vertices property that neesd to be defined as a sequence of `Point2D` instances. So here, not only do we want the `vertices` attribute of our `Polygon`to be an iterable of some kinf, we also want the elements to all be instances of the `Point2D` class. In turn we'll also wanto to make sure that coordinates for `Point2D` are non-negative integer values (as might be expected in computer screen coordinates)

In [635]:
class Int:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self.prop_name} must be a integer.')
        
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'{self.prop_name} must be at lest {self.min_value}')
        
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.prop_name} cannot be greater than {self.max_value}")
        
        instance.__dict__[self.prop_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        
        return instance.__dict__.get(self.prop_name, None)

In [636]:
class Point2D:
    x = Int(min_value=0, max_value=800)
    y = Int(min_value=0, max_value=600)

    def __init__(self, x, y):
        self.x = x
        self.y = y 

    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __eq__(self, other):
        return isinstance(other, Point2D) and self.x == other.x and self.y == self.y
    
    def __hash__(self):
        return hash(f'{self.x}, {self.y}')

In [637]:
p = Point2D(0, 10)

In [638]:
p.x, p.y

(0, 10)

In [639]:
p

Point2D(x=0, y=10)

In [640]:
str(p)

'(0, 10)'

In [641]:
try:
    p = Point2D(0, 900)
except ValueError as ve:
    print(ve)

y cannot be greater than 600


### Collections

In [642]:
import collections

In [643]:
isinstance([1, 2, 3], collections.abc.Sequence)

True

In [644]:
isinstance((1,2,3), collections.abc.Sequence)

True

In [645]:
isinstance({1, 2, 3}, collections.abc.Sequence)

False

In [646]:
isinstance([1, 2, 3], collections.abc.MutableSequence)

True

In [647]:
isinstance((1,2,3), collections.abc.MutableSequence)


False

In [648]:
class Point2DSequence:
    def __init__(self, min_len=None, max_len=None):
        self.min_len = min_len
        self.max_len = max_len

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name

    def __set__(self, instance, value):
        if not isinstance(value, collections.abc.Sequence):
            raise ValueError(f'{self.prop_name} must be a sequence type')
        
        if self.min_len is not None and len(value) < self.min_len:
            raise ValueError(f'{self.prop_name} must be at least {self.min_len} elements long')
        
        if self.max_len is not None and len(value) > self.max_len:
            raise ValueError(f'{self.prop_name} must not exceed {self.max_len} elements')
        
        for index, item in enumerate(value):
            if not isinstance(item, Point2D):
                raise ValueError(f'Item {item} at index {index} is not Point2D instance')
            
        instance.__dict__[self.prop_name] = list(value) # transform in a mutable sequence 

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        
        if self.prop_name not in instance.__dict__:
            instance.__dict__[self.prop_name] = []

        return instance.__dict__[self.prop_name]

In [649]:
class Polygon:
    vertices = Point2DSequence(min_len=3)

    def __init__(self, *vertices):
        self.vertices = vertices

In [650]:
p = Polygon(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0))


In [651]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [652]:
p.__dict__['vertices']

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [653]:
class Polygon:
    vertices = Point2DSequence(min_len=3)

    def __init__(self, *vertices):
        self.vertices = vertices

    def append(self, point):
        if not isinstance(point, Point2D):
            raise ValueError(f'Can only append Point2D instances')
        max_len = type(self).vertices.max_len
        
        if max_len is not None and len(self.vertices) >= max_len:
            raise ValueError(f'Vertices len is at max: {max_len}')

        self.vertices.append(point)

    def __len__(self):
        return len(self.vertices)
    
    def __getitem__(self, idx):
        return self.vertices[idx]
    
    def __iadd__(self, pt):
        self.append(pt)

    def __contains__(self, pt):
        return pt in self.vertices


In [654]:
p = Polygon(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0))
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [655]:
p.append(Point2D(10, 0))

In [656]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0), Point2D(x=10, y=0)]

And what if I want to implement specific shapes of polygons without copy and paste code?

In [657]:
class Triangle(Polygon):
    vertices = Point2DSequence(min_len=3, max_len=3)

In [658]:
t = Triangle(Point2D(0,1), Point2D(1,0), Point2D(2,0))

In [659]:
t.vertices

[Point2D(x=0, y=1), Point2D(x=1, y=0), Point2D(x=2, y=0)]

In [660]:
t.append(Point2D(2, 1))

ValueError: Vertices len is at max: 3

In [661]:
p = Polygon(Point2D(0,0), Point2D(1,0), Point2D(1,1))
p.vertices

[Point2D(x=0, y=0), Point2D(x=1, y=0), Point2D(x=1, y=1)]

In [662]:
len(p)

3

In [663]:
p[1]

Point2D(x=1, y=0)

In [664]:
p[1:3]

[Point2D(x=1, y=0), Point2D(x=1, y=1)]

In [665]:
list(p)

[Point2D(x=0, y=0), Point2D(x=1, y=0), Point2D(x=1, y=1)]

In [666]:
Point2D(0,0) in p 

True

whatch out bc if `__eq__` is not implremented yet, python is going to use the hash id for comparision, not the point value in this example